In [30]:
import pandas as pd
import os
cwd = os.getcwd()
year = 5
df1 = pd.read_excel(cwd + '/Retail Turnover State by Industry Subgroup Original.xlsx', sheet_name='Data1')
df2 = pd.read_excel(cwd + '/Formula Sheet.xlsx', sheet_name='Industry Manager Title')
df3 = pd.read_excel(cwd + '/Formula Sheet.xlsx', sheet_name='Manager Annual Wage')

df1.drop(columns=['Unnamed: 0'], inplace=True)
df1 = (
    df1.tail(year * 12 + 1)
        .pct_change(fill_method=None)
        .dropna(how='all')
        .mean()
        .to_frame(name='Turnover Rate')  # Name the value column
        .reset_index()     # Name the former index as a column
)

# Now split the 'index' column into parts
df1[['Part1', 'State', 'Industry','Part2']] = df1['index'].str.split(';', expand=True)

# Clean spaces
df1['State'] = df1['State'].str.strip()
df1['Industry'] = df1['Industry'].str.strip()

# Drop the useless columns
df1 = df1[['State', 'Industry', 'Turnover Rate']]
df1 = df1.merge(df2, how='left', left_on='Industry', right_on='Retail Trade')
df3 = df3.rename(columns={'State/Territory': 'State'}).melt(id_vars='State', var_name='Title', value_name='Salary')
df1 = df1.merge(df3_long, how='left', on=['State', 'Title'])

In [31]:

with pd.ExcelWriter(cwd + '/Formula Sheet.xlsx', mode='a', engine='openpyxl') as writer:
    if 'Industry' in writer.book.sheetnames:
        del writer.book['Industry']
    df1.to_excel(writer, sheet_name='Industry Metrics', index=False)